## What is COCO

COCO is a large-scale object detection, segmentation, and captioning dataset. COCO has several features like Object segmentation, Recognition in context, 80 object categories, over 200k labeled images and 1.5 million labeled object instances. Learn more here: http://cocodataset.org/#home

The COCO dataset comes with its very own label format for each of the label categories: Detection, Captioning, Keypoints, Stuff, Panoptic. The COCO import function within the Datagym Python Package currently supports Detection and Captioning.

## Connecting and setting up

Lets get started by importing our Datagym client and the Coco importer

In [3]:
from datagym import Client
from datagym.importers.coco import Coco

First we must define which project and dataset we will be working with. The project needs to be manually set up in your browser. 

In [5]:
PROJECT_NAME = "Coco"
DATASET_NAME = "coco_val2017"

Connect with your personal API key and get your project by name

In [3]:
client = Client(api_key="<API_KEY>")
project = client.get_project_by_name(project_name=PROJECT_NAME)

Create new Dataset, here we wrap the dataset creation in a try/except clause in case the dataset is already created

In [4]:
import warnings
from datagym import ClientException
try:
    dataset = client.create_dataset(name=DATASET_NAME, short_description="Images from the coco validation set 2017")
except ClientException as e:
    warnings.warn(e.args[0])

Now we create our list of image urls that we want to upload to our dataset. This list of urls won't necessarily be the same for you, depending where you have saved your images. In this case we chose to create a list of the coco_urls that are provided within the coco json files. 

In [5]:
import json

image_url_list = []
local_coco_json_path = "PATH_TO_instances_YEAR.json"  # .../instances_val2017.json for example

with open(local_coco_json_path) as json_file:
    data = json.load(json_file)

for image in data["images"]:
    image_url_list.append(image["coco_url"])

print("Number of URLs: {}".format(len(image_url_list)))
print("Example URL: {}".format(list(image_url_list)[0]))

Number of URLs: 5000
Example URL: http://images.cocodataset.org/val2017/000000397133.jpg


Here we are choosing to upload the first 200 coco images from the url list.

In [6]:
amount_images = 200

dataset = client.get_dataset_by_name(dataset_name=DATASET_NAME)
upload_results = client.create_images_from_urls(dataset_id=dataset.id, image_url_list=image_url_list[:amount_images])
print(f"{amount_images} Images uploaded")

Uploading image urls: [########################################] 4/4

200 Images uploaded


In case the dataset is not already attached to the project, this is done here. Again wrapped in try/except in case they are already connected to one another. 

In [7]:
project = client.get_project_by_name(project_name=PROJECT_NAME)
dataset = client.get_dataset_by_name(dataset_name=DATASET_NAME)

from datagym import APIException
try:
    client.add_dataset(dataset_id=dataset.id, project_id=project.id)
except APIException as e:
    warnings.warn(e.args[0])

Create image id dictionary that returns the internal datagym media id when given an image name

In [8]:
image_ids_dict = dict()

for image in dataset.images:
    image_ids_dict[image.image_name] = image.id
    
print(f'One example from the image_ids_dict:\n {{"{image.image_name}" : "{image_ids_dict[image.image_name]}"}}')

One example from the image_ids_dict:
 {"000000458790.jpg" : "9f52a33a-ef09-468b-a227-a35de8519bda"}


Load coco labels from teh respective jsons. 

__coco.add_object_detection_data__ is used for all instances__**__.json.

The when using the add_object_detection_data method you can either choose to upload the bounding box or the polygon containing the object.

__coco.add_captions_data__ is used for all captions__**__.json.

In [9]:
coco = Coco()

with open("PATH_TO_instances_YEAR.json") as json_file:    # .../instances_val2017.json for example
    instances_data = json.load(json_file)
with open("PATH_TO_captions_YEAR.json") as json_file:    # .../captions_val2017.json for example
    captions_data = json.load(json_file)

coco.add_object_detection_data(instances_data, polygon=False)
coco.add_captions_data(captions_data)

Now that we have added all the relevant labels from their respective json files, we are ready to convert them into the uploadable datagym json file. Here we will also print the first entry.

In [10]:
upload_dict = coco.get_datagym_label_dict(image_ids_dict)

import pprint
pprint.pprint(upload_dict[0])

{'global_classifications': {'caption': ['A table with pies being made and a '
                                        'person standing near a wall with pots '
                                        'and pans hanging on the wall.']},
 'internal_media_ID': '715cc778-df03-4ff9-898d-63c6d502e975',
 'keepData': False,
 'labels': {'appliance': [{'classifications': {'subcategory': ['oven']},
                           'geometry': [{'h': 98.37,
                                         'w': 192.56,
                                         'x': 1.36,
                                         'y': 164.33}]},
                          {'classifications': {'subcategory': ['oven']},
                           'geometry': [{'h': 98.98,
                                         'w': 191.36,
                                         'x': 0.0,
                                         'y': 210.9}]},
                          {'classifications': {'subcategory': ['sink']},
                           'geometr

Before we can upload the data to datagym we need to set up the label configuration to represent the label hierarchies of coco. 

In [11]:
from datagym.models.label_config import LabelConfig
label_config = LabelConfig()

label_config.add_freetext(entry_key="caption", entry_value="caption")

annotations = coco.category_dict

categories = set([annotation['supercategory'] for _, annotation in annotations.items()])

for val in categories:
    
    subcategories_options_dict = {annotation['name']:annotation['name'] for _, annotation in annotations.items() if annotation['supercategory']==val}
    
    geometry = label_config.add_rectangle(entry_key=val, entry_value=val)
    geometry.add_select(entry_key="subcategory", entry_value="subcategory", options_dict=subcategories_options_dict)
    

    
response = client.upload_label_config(project.label_config_id, label_config.toJson())

With your Label configuration successfully set up you are now ready to upload your coco labels.

In [12]:
project = client.get_project_by_name(project_name=PROJECT_NAME)

errors = client.import_label_data(project_id=project.id, label_data=upload_dict)

Uploading annotations: [########################################] 4/4

